In [27]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [47]:
import warnings
import pandas as pd
import numpy as np
import pickle
import sys
import time
import glob
import re
import os.path
 
from tqdm.notebook import trange, tqdm
import tweepy
# from sklearn.model_selection import cross_validate
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import classifier_utils as cu

from plotly.offline import init_notebook_mode
import plotly.offline as py
import plotly.graph_objs as go
init_notebook_mode(connected=True)
warnings.filterwarnings(action='once')

In [48]:
def load_user_tweets(directory, limit=None):
    user_tweets = {}
    i = 0
    for f in glob.glob(directory + '/*.p'):
        uid = int(re.findall(r'\d+', f.replace(directory, ''))[0])
        with open(f, 'rb') as file:
            tweets = pickle.load(file)
        texts = [t.text for t in tweets if not hasattr(t, 'retweeted_status')]
        user_tweets[uid] = texts
        i += 1
        if limit is not None and i > limit:
            break
    return user_tweets

def load_user_tweets_by_name(directory, usernames):
    user_tweets = {}
    for u in usernames:
        path = directory + '/' + u + '.p'
        if not os.path.isfile(path):
            continue
        with open(path, 'rb') as file:
            tweets = pickle.load(file)
        texts = [t.text for t in tweets if not hasattr(t, 'retweeted_status')]
        user_tweets[u] = texts
    return user_tweets

In [28]:
1

1

In [ ]:
user_tweets = load_user_tweets('../data/BBlevel1_user_tweets', limit=5000)

In [ ]:
naziblkr_user_tweets = load_user_tweets('../data/NaziBlocker_user_tweets/', limit=5000)

In [53]:
control_user_tweets = load_user_tweets('../data/control_user_tweets2', limit=5000)

In [78]:
senete_harrasing_user_tweets = load_user_tweets_by_name('../data/harassers_from_targetted', usernames=harrassing_users_list)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: ResourceWarning:

unclosed <ssl.SSLSocket fd=71, family=AddressFamily.AF_INET, type=2049, proto=6, laddr=('128.238.147.119', 34534), raddr=('104.244.42.130', 443)>

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: ResourceWarning:

unclosed <ssl.SSLSocket fd=39, family=AddressFamily.AF_INET, type=2049, proto=6, laddr=('128.238.147.119', 34536), raddr=('104.244.42.130', 443)>

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: ResourceWarning:

unclosed <ssl.SSLSocket fd=41, family=AddressFamily.AF_INET, type=2049, proto=6, laddr=('128.238.147.119', 34538), raddr=('104.244.42.130', 443)>

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: ResourceWarning:

unclosed <ssl.SSLSocket fd=47, family=AddressFamily.AF_INET, type=2049, proto=6, laddr=('128.238.147.119', 34544), raddr=('104.244.42.130', 443)>

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: ResourceWarning

In [6]:
p = pickle.load(open('../models/model_toxic_comments_dataset_severe_toxic.p', 'rb'))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: ResourceWarning:

unclosed file <_io.BufferedReader name='../models/model_toxic_comments_dataset_severe_toxic.p'>



In [ ]:
naziblkr_user_scores = {}
for uid, tweets in naziblkr_user_tweets.items():
    if len(tweets) < 500 or uid in user_scores:
        continue
    preds = p.predict_proba(tweets)[:, 1]
    
    frac = (preds > 0.8).sum() / len(preds)
    if frac > 0.01:
        print(np.array(tweets)[preds > 0.8])
    naziblkr_user_scores[uid] = frac

In [ ]:
# user_scores = {}
for uid, tweets in tqdm(user_tweets.items()):
    if len(tweets) < 500 or uid in user_scores:
        continue
    preds = p.predict_proba(tweets)[:, 1]
    
    frac = (preds > 0.8).sum() / len(preds)
    if frac > 0.01:
        print(np.array(tweets)[preds > 0.8])
    user_scores[uid] = frac

In [54]:
control_user_scores = {}
for uid, tweets in control_user_tweets.items():
    if len(tweets) < 500 or uid in control_user_scores:
        continue
    preds = p.predict_proba(tweets)[:, 1]
    
    frac = (preds > 0.8).sum() / len(preds)
    if frac > 0.01:
        print(uid, frac, np.array(tweets)[preds > 0.8])
    control_user_scores[uid] = frac

/usr/lib/python3.6/re.py:212: FutureWarning:

split() requires a non-empty pattern match.



1084592826917310469 0.013811123553564763 ['@pjmsexyy fucking i' '@yvesaintseokjin FUCKING HES'
 'WHAT THE FUCK IS IT DOING CMON'
 'HERE WE FUCKING GO https://t.co/o0YV8jgnn1'
 'BOYZ WITH FUN IS ON EVERYONE SHUT THE FUCK UP'
 'fuck you park jimin. you know i never really thought i’d have to tweet this but i cant believe this bitch has the r… https://t.co/svJtIUBep5'
 '@rainjinsol RIGHT* YOU SUCK AT SPELLING'
 'ATEEZ IS PLAYING AT MAMA HOLY FUCK FUCK FUCK'
 'the PRETTIEST fucking human shut up https://t.co/sXA2BE6iIJ'
 'why did i laugh so fucking hard omg https://t.co/HoRMyuL6Yg'
 'IVE BECOME SL FUCKINF SOFT WHAT THE FUCK DO I DK NOW'
 'don’t mind me while i lose my shit ab how fucking adorable my baby is https://t.co/hyE6N4i0Js'
 'i cant watch bv until THIS WEEKEND WHAT THE FUCK'
 'can jimin pls post i MISS HIM WHAT THE FUCK NAJFNS'
 'yea ok fuck this i’m going to bed.'
 '@CHlMSJJK YES WHAT THE FUCKING FUCK'
 '@CHlMSJJK fuck u, crackers r mines'
 '@CHlMSJJK NO WHAT THE FUCK U WAS WXCJRE

179012067 0.018948365703458078 ['This my shit 🤣🤣 https://t.co/SuH4Ewq4Lf' 'Watch my shit for whet ?'
 'Y’all be retweet some funny as shit ... keep it the fuck 🤣💪🏾'
 'K camp new shit 🔥'
 'My girl out hiking \U0001f97e n shit ...... jealous af 🙄'
 'It’s hot ass hell in Pleasanton 😓 sheeesh bitch .'
 '🤣🤣🤣 fuck , this my bitch 4L https://t.co/hqsesyKgrM'
 'Kevin gates new shit 🔥🔥🔥'
 'I really dislike @alliirose_  bitch ass rn 🙏🏾... on mamas 🤣🤣'
 '@alliirose_ Biiiiiitttchhh 🤣🤣 fuck you'
 '😂 he’s really fucking sucking on the screen https://t.co/QxUUjo5Jxt'
 'This my fucking shit https://t.co/noHjLWl7s1'
 'I’m fuck starving bitch , I left my funds at the house 🤦🏾\u200d♀️'
 "2 more hours then I'm off ... I can't wait bruh , a bitch is fucking starving 🙃"
 "@hennybuxaplenty somebody need put on a stamp on it &amp; send this bitch off cause we're fucking mastering minds 💡💡"
 'Who trynna fuck around tonight?? 🍻'
 '🤣🤣 Got yo ass , mike/Gary u lil dirty man bitch  https://t.co/9lQHL5XN2S'
 'Cause

286406209 0.01365328920148945 ['@savemadebyjarry god i am so LUCKYYYY you are a fucking queen 😭💕'
 '@savemadebyjarry we are so fucking cute. you are so fucking cute. i miss you so MUCH oh my GOD THIS IS SO SAD'
 'BRO WHAT THE FUCK \n\nI LOOOOOOOVE SCRUBS https://t.co/oUcvwidsMi'
 '@iwillbutcheryou i fucking hate you'
 'FIRE JOHN HYNES INTO ORBIT HOLY FUCKING SHIT I AM SO S I C K AND T I R E D OF THIS TALENTED TEAM LOOKING DAZED AND… https://t.co/8vCNBdt8p8'
 '@savemadebyjarry ❗️thats my girl holy shit ❗️'
 'holy fucking shit. \n\nthe bald man did it. https://t.co/ChM3wWNlc4'
 '@NJDevils LETS FUCKING GO BOYSSSSSSSSDDD'
 'my god.... they fucking did. THEY DID IT. GOOD LINES. https://t.co/iHnXG5hzjO'
 'goodbye you fucking cheeto https://t.co/0b0QAkgEbV'
 'LETS FUCKING GO BABY!!!! https://t.co/gurGOPQ77I'
 '@nhlvictoria you fucking suck you sucker'
 'LETS FUCKING GO BOYS OH MY GODDDDDD https://t.co/qKNyW52Z8O'
 '@QuiteAligned shit the fuck up Chrysler trash'
 '@QuiteAligned really fucking 

45354512 0.018223234624145785 ['What a sad fucking bitch and a waste of good air.\U0001f92c\U0001f92c https://t.co/MVMtSVDWWc'
 'Fucking cruel bastards https://t.co/IoXXrdSpLz'
 'Lowlife piece of shit. https://t.co/dYakrhs6nB'
 'This is fucking disgusting. Breaks your heart. 💔💔😭 https://t.co/T5MFYY2qTb'
 'Lowlife piece of shit! https://t.co/TMndqG9rbV'
 'What a fucking lowlife piece of shit! https://t.co/dQGL91Zpkl'
 'Has this moronic piece of shit been charged? https://t.co/0qBhK19fo1'
 'Moronic, lowlife piece of shit. \U0001f92c\U0001f92c\U0001f92c\U0001f92c https://t.co/CbM7JhEdU5'
 'Still a cruel fucking bitch though. https://t.co/8axkXm9eKD'
 'Sick doesn’t describe the evil piece of shit. https://t.co/9kHd9zRNQB'
 'Fucking lowlife piece of shit.  \U0001f92c\U0001f92c\U0001f92c https://t.co/EaqQruvR6i'
 'Oh he fucking will, will he? What a twat. \U0001f92c https://t.co/YzryRF9VmP'
 'Disgusting piece of lowlife shit. \U0001f92c\U0001f92c\U0001f92c https://t.co/6NHoTnsNlg'
 'Fucking 

...

726566276 0.017941952506596307 ['@extralargepeas Fuck'
 'I’m bored as fuck and I feel like shit. Entertain me'
 'I feel fucking grim today' 'I’m fucking dead https://t.co/PK8rwNRjBs'
 '@alexanderianx No fucking way' 'I’m a fucking dumb bitch'
 'Fuck https://t.co/AgGbUKq0qa'
 'Just fucking done my tan amd I’m crying it all away https://t.co/rcGWg54UMU'
 'UP \n\nTHE FUCKING\n\nREDS ❤️❤️❤️' 'I’m so fucking knackered'
 '@caragreen__x @mayapologies Fuck offfff 😂😂😂😂😂😂😂😂😂😂'
 'That’s the fucking cutest thing I’ve ever seen https://t.co/sYAbwnyINc'
 '‘That’s a fucking forward roll you fucking spastic’'
 'Take that back I’m a fucking idiot https://t.co/y5ayFrFuXD'
 'I’m so fucking funny' '@b14kepriv Go fuck yourself' 'Fuck my life'
 'Too fucking right https://t.co/TM2hrOYrej'
 'Dis bitch has royally fucked her back up'
 '@sofaritsian Fuck off \n\nSEVEN YEARS?!'
 'Marks and Spencer go fuck yourselves https://t.co/R7dsZMSV2v'
 'I fucking hate today it’s been such a shit day I’m going to bed'
 '@_J

965004506 0.012125079770261647 ['She don’t gotta have ass imma cop her the fake shit'
 'Shut your bitch ass up https://t.co/uGhmP4FYlA'
 'Fuck it roll another one' 'Fuck you https://t.co/kS72wJQaJG'
 'Bitches ain’t shit 🤦🏾\u200d♂️'
 'I say bitches ain’t shit &amp; I’m misogynistic... y’all say niggas ain’t shit &amp; it’s “Periodt Pooh”... I hate it here'
 'Bitches is really bums shit wild'
 'Man shut the fuck up https://t.co/Yf0SUfC0XB'
 '@lovemeleniexo Fuck boffum 🤷🏾\u200d♂️' 'Fuck Christopher Columbus'
 'Fuck all that shit... https://t.co/loeNmgr2TY'
 'The bitch got 10 years!!!! Probably gonna do 5-7 for KILLING A MAN.... Man fuck this country on God'
 'I’ll fuck a fat bitch if she know how to get money'
 'Fuck football lmao https://t.co/v0L5LshLvm'
 'Boy shut the fuck up 🤣🤣🤣 https://t.co/On5bPP2fg2'
 'Fuck around &amp; air a pussy nigga out like linen' 'Kickin shit Ryuken'
 'Shut the fuck up https://t.co/0pB4L83gpM'
 'Fuck 12 😒 https://t.co/IWVR73j9tP']
2427908371 0.017492711370262

1480452170 0.027777777777777776 ['Fuck Farm workers'
 'Also family is fake as fuck? And all they do is bitch, complain, and make fun of you?? https://t.co/yoyp2QLpnq'
 'All this shit about the colored cosplayer; WHO FUCKING CARES!!!!! THEY ARE ********FICTIONAL**********'
 'People are SO sensitive jesus fucking Christ'
 'You’re a fucking crackhead https://t.co/Zco60uw0ND'
 'FUCK YOU NEW YORK STATE AND EVERYONE THAT RUNS IT. \n\nWHY THE FUCK DO I HAVE TO PAY FOR THESE PEOPLE WHEN I LITERALL… https://t.co/iTtXFov0Q1'
 'Goddamn if you REALLY wanna get fucked up? Get drunk and take a fucking dab'
 'Lol shit I’m fucking faded' 'God I can’t wait until I’m fucking dead'
 'I’m fucking faded'
 'Oh my god the sound of a crying baby makes me wanna go fucking insane'
 'I’m fucking hungry' 'What the FUCK' 'fucking idiot'
 '@UchihaBitch_ Damn spoiler as fuck'
 '@tonyposnanski Fuck around with a beehive without protective gear'
 'Eat my fucking ass you dumb bitch'
 'you’re fucking weird for letting y

Exception ignored in: 'zmq.backend.cython.message.Frame.__dealloc__'
Traceback (most recent call last):
  File "zmq/backend/cython/checkrc.pxd", line 12, in zmq.backend.cython.checkrc._check_rc (zmq/backend/cython/message.c:4379)
KeyboardInterrupt: 


KeyboardInterrupt: 

In [51]:
senete_harrasing_user_scores = {}
for uid, tweets in senete_harrasing_user_tweets.items():
    if len(tweets) < 500 or uid in senete_harrasing_user_scores:
        continue
    preds = p.predict_proba(tweets)[:, 1]
    
    frac = (preds > 0.8).sum() / len(preds)
    if frac > 0.01:
        print(np.array(tweets)[preds > 0.8])
    senete_harrasing_user_scores[uid] = frac

/usr/lib/python3.6/re.py:212: FutureWarning:

split() requires a non-empty pattern match.



['@SunsetFranklin Fuck off' '@MikeNealIN Oh fuck off you dunce'
 '@tombrennerphoto @realDonaldTrump What a worthless piece of shit'
 '@JessicaASmith8 @SenatorBraun @SenatorShaheen @YahooFinance Hard fucking pass'
 '@indgop @SenatorBraun Hard fucking pass' '@marinehel Fuck off'
 '@washingtonpost Because she’s a fucking liar' '@RobMKendall Fuck off'
 '@TheMopLady Fuck that guy forever'
 '@SenatorBraun @Varneyco @FoxBusiness Fuck you'
 '@RaheemKassam @FireRokita @SenatorBraun @JasonMillerinDC You are a fucking Nazi.'
 '@SenatorBraun @JasonMillerinDC @RaheemKassam Garbage loves garbage. You are a fucking disgrace.'
 '@Citiz4Solutions @SenatorBraun I’ll send him a fucking message https://t.co/E7cxXBek0K'
 '@VAVetBenefits @SebGorka What the fuck? Seb’s a fucking Nazi you tools.'
 '@CLH45 Hey- fuck off! Fuck @IvankaTrump too!'
 '@rmittpanda What the fuck do you think you are saying? You’re an idiot'
 '@megynkelly Hardest of hard fucking passes. Please go away forever'
 '@MandalorianAlex @hwkb

In [55]:
py.iplot([
#     go.Histogram(x=[u for u in user_scores.values()], name='BlockBot L1', histnorm='probability'),
#     go.Histogram(x=[u for u in naziblkr_user_scores.values()], name='Nazi Blocker', histnorm='probability'),
    go.Histogram(x=[u for u in senete_harrasing_user_scores.values()], name='Senete Harassers', histnorm='probability'),
    go.Histogram(x=[u for u in control_user_scores.values()], name='Control', histnorm='probability'),
])

In [ ]:
py.iplot([
    go.Histogram(x=[len(u) for u in user_tweets.values()], name='BlockBot L1', histnorm='probability'),
    go.Histogram(x=[len(u) for u in control_user_tweets.values()], name='Control', histnorm='probability'),
])

In [7]:
senate_target_users = [
    'SenHydeSmith',
    'SenTinaSmith',
    'gillibrandny',
    'SenJackyRosen',
    'SenCortezMasto',
    'SenKamalaHarris',
    'SenMcSallyAZ',
    'SenJoniErnst',
    'SenatorSinema',
    'SenatorBaldwin',
    'SenatorFischer',
    'SenDuckworth',
    'SenWarren',
    'SenatorHassan',
    'SenFeinstein',
    'PattyMurray',
    'MarshaBlackburn',
    'SenCapito',
    'SenatorCantwell',
    'SenatorShaheen',
    'maziehirono',
    'SenStabenow',
    'SenAmyKlobuchar',
    'SenatorCollins',
    'lisamurkowski'
]

In [11]:
harrassing_users = {}
for u in senate_target_users:
    with open('../data/target_users/' + u + '.p', 'rb') as f:
        print(u)
        targetted_user_tweets = pickle.load(f)
        tweets = [t.text for t in targetted_user_tweets]
        users = [t.user.screen_name for t in targetted_user_tweets]
        preds = p.predict_proba(tweets)[:, 1]
        print(np.array(tweets)[preds > 0.7])
        
        h_users = np.unique(np.array(users)[(preds > 0.7)])
        if len(h_users) > 0:
            harrassing_users[u] = h_users
        

SenHydeSmith


/usr/lib/python3.6/re.py:212: FutureWarning:

split() requires a non-empty pattern match.



['@SenHydeSmith is another piece of shit GOP bought off by the NRA. She blocked a gun bill WHILE this shooting occurr… https://t.co/rK8I1ECBIY']
SenTinaSmith
[]
gillibrandny
[]
SenJackyRosen
['@SenJackyRosen Look at this, no fear spouting Harry Reids shit']
SenCortezMasto
[]
SenKamalaHarris
["RT @PieMan9669: @SenKamalaHarris Are you saying Native men murder at 10x the national average? That's some racist shit."
 "@SenKamalaHarris Are you saying Native men murder at 10x the national average? That's some racist shit."]
SenMcSallyAZ
[]
SenJoniErnst
['@IWF @SenJoniErnst And it/you ignore Male victims.\nVAWA is feminist bull shit.'
 '@ColMorrisDavis @realDonaldTrump Actually, he’s just BAT SHIT CRAZY.💩\n@realDonaldTrump @WhiteHouse @whca @FLOTUS… https://t.co/VtO9hefM53']
SenatorSinema
[]
SenatorBaldwin
[]
SenatorFischer
[]
SenDuckworth
['@llilly @SenDuckworth @EsperDoD @realDonaldTrump @DeptofDefense THEN WATCH HIS SHIT AGAIN. WHILE YOU CODDLE OUR ENE… https://t.co/j9XJMToeID']
SenWarren
[

In [15]:
harrassing_users_list = []
for users in harrassing_users.values():
    harrassing_users_list += list(users)

In [17]:
pickle.dump((harrassing_users, harrassing_users_list), open('../data/senete_harrassing_users.p', 'wb'))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: ResourceWarning:

unclosed file <_io.BufferedWriter name='../data/senete_harrassing_users.p'>



In [76]:
nyt_users = pickle.load(open('../data/female_dailymail.p', 'rb'))
nyt_users = [u.screen_name for u in nyt_users]

harrassing_users = {}
for u in nyt_users:
    path = '../data/target_users/' + u + '.p'
    if not os.path.isfile(path):
        continue
    with open(path, 'rb') as f:
        print(u)
        targetted_user_tweets = pickle.load(f)
        if len(targetted_user_tweets) == 0:
            continue
        tweets = [t.text for t in targetted_user_tweets]
        users = [t.user.screen_name for t in targetted_user_tweets]
        preds = p.predict_proba(tweets)[:, 1]
        print(np.array(tweets)[preds > 0.7])
        
        h_users = np.unique(np.array(users)[(preds > 0.7)])
        if len(h_users) > 0:
            harrassing_users[u] = h_users
        

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: ResourceWarning:

unclosed file <_io.BufferedReader name='../data/female_dailymail.p'>



In [77]:
nyt_users

['health_kate',
 'health_justine',
 'IsabelOakeshott',
 'RealLizJones',
 'katienicholl',
 'lizmavor',
 'EMD1990',
 'MelanieLatest',
 'TamaraAbraham',
 'Emine_Sinmaz',
 'AlexisParrDiary',
 'larisamlbrown',
 'J_Reilly',
 'L_M_Davis',
 'LindsArchibald',
 'RE_DailyMail',
 'byRebeccaEvans',
 'louise_eccles',
 'lmwilliamson7',
 'the_topspin',
 'Jenny_Stanton',
 'Lucy_Osborne',
 'bianca_london',
 'IonaKirby',
 'kerushton',
 'RuthLythe',
 'anneshooter',
 'VBischoff',
 'KatherinThomson',
 'marthakelner',
 'MyraButterworth',
 'suzanne_moore',
 'deborahross',
 'laraking',
 'k_faulkner',
 'jolienovak']

In [63]:
harrassing_users_list = []
for users in harrassing_users.values():
    harrassing_users_list += list(users)

In [65]:
pickle.dump((harrassing_users, harrassing_users_list), open('../data/nyt_harrassing_users.p', 'wb'))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: ResourceWarning:

unclosed file <_io.BufferedWriter name='../data/nyt_harrassing_users.p'>



In [ ]:
tweets = [t.text for t in targetted_user_tweets]
users = [t.user.screen_name for t in targetted_user_tweets]
preds = p.predict_proba(tweets)[:, 1]

In [ ]:
np.array(preds)[(preds > 0.5)]

In [ ]:
np.array(users)[(preds > 0.5)]

In [ ]:
np.array(tweets)[(preds > 0.5)]


In [66]:
auth = tweepy.OAuthHandler('QLbvBAuKCD9GOSMzM4ly3Jueb', '9qX5Suoeuoyccict1tkbCheJC42djHrIu71pL0GYnxw1DEhSSp')
auth.set_access_token('430684463-hPPEVIKcDIiLGqxw37XuYRgvXNxIBXHO9NZeYz3o', 'GZ8K5u3SLjdWvXPHSrgfWfyvpAkfKLr8csFo80UBECZ5h')
api = tweepy.API(auth)

def limit_handled(cursor):
    while True:
        try:
            yield cursor.next()
        except tweepy.RateLimitError:
            time.sleep(15 * 60)
        except tweepy.TweepError as e:
            print(e)
            time.sleep(15 * 60)

def get_all_tweets(screen_name):
    alltweets = []
    new_tweets = api.user_timeline(screen_name=screen_name,count=200)

    #save most recent tweets
    alltweets.extend(new_tweets)
    if len(alltweets) == 0:
        return alltweets
    
    #save the id of the oldest tweet less one
    oldest = alltweets[-1].id - 1

    #keep grabbing tweets until there are no tweets left to grab
    while len(new_tweets) > 0:
        print("getting tweets before %s" % (oldest))

        #all subsiquent requests use the max_id param to prevent duplicates
        new_tweets = api.user_timeline(screen_name = screen_name,count=200,max_id=oldest)

        #save most recent tweets
        alltweets.extend(new_tweets)

        #update the id of the oldest tweet less one
        oldest = alltweets[-1].id - 1

        print("...%s tweets downloaded so far" % (len(alltweets)))
    return alltweets
            
def get_mention_tweets(username, limit=4000):
    statuses = []
    with tqdm(total=limit) as pbar:
        for status in limit_handled(tweepy.Cursor(api.search, q='@' + username).items()):
            if(len(statuses)) > limit:
                break
            statuses.append(status)
            pbar.update(1)
            
    return statuses

In [71]:
scanned_users = [f.replace('../data/harassers_from_targetted/', '').replace('.p', '') for f in glob.glob("../data/harassers_from_targetted/*.p")]

In [68]:
failed = []

In [72]:
for user in harrassing_users_list:
    if user in scanned_users:
        continue
    if user in scanned_users or user in failed:
        continue
    try:
        tweets = get_all_tweets(user)
        pickle.dump(tweets, open('../data/harassers_from_targetted/' + user + '.p', 'wb'))
    except tweepy.RateLimitError:
        time.sleep(15 * 60)
    except Exception as e:
        print(e)
        failed.append(user)

getting tweets before 1193196869809295359
...398 tweets downloaded so far
getting tweets before 1182054392607977471
...597 tweets downloaded so far
getting tweets before 1167622015991390208
...795 tweets downloaded so far
getting tweets before 1151935132934320128
...993 tweets downloaded so far
getting tweets before 1133034573011587072
...1193 tweets downloaded so far
getting tweets before 1125218107952443392
...1387 tweets downloaded so far
getting tweets before 1116841812919050239
...1584 tweets downloaded so far
getting tweets before 1107689427550244869
...1776 tweets downloaded so far
getting tweets before 1096870879592673286
...1976 tweets downloaded so far
getting tweets before 1076029254477987839
...2173 tweets downloaded so far
getting tweets before 1024418515082772480
...2371 tweets downloaded so far
getting tweets before 1000481856268197887
...2570 tweets downloaded so far
getting tweets before 968937763612524544
...2763 tweets downloaded so far
getting tweets before 95910612

/usr/local/lib/python3.6/dist-packages/tweepy/models.py:44: ResourceWarning:

unclosed <ssl.SSLSocket fd=61, family=AddressFamily.AF_INET, type=2049, proto=6, laddr=('128.238.147.119', 34506), raddr=('104.244.42.130', 443)>

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: ResourceWarning:

unclosed file <_io.BufferedWriter name='../data/harassers_from_targetted/Derek_IB105.p'>

/usr/local/lib/python3.6/dist-packages/tweepy/binder.py:222: ResourceWarning:

unclosed <ssl.SSLSocket fd=62, family=AddressFamily.AF_INET, type=2049, proto=6, laddr=('128.238.147.119', 34508), raddr=('104.244.42.130', 443)>



getting tweets before 1201205903879487489
...400 tweets downloaded so far
getting tweets before 1200536374404943878


/usr/lib/python3/dist-packages/simplejson/decoder.py:400: ResourceWarning:

unclosed <ssl.SSLSocket fd=62, family=AddressFamily.AF_INET, type=2049, proto=6, laddr=('128.238.147.119', 34512), raddr=('104.244.42.130', 443)>



...599 tweets downloaded so far
getting tweets before 1199912460125425666
...799 tweets downloaded so far
getting tweets before 1199544519328456703
...999 tweets downloaded so far
getting tweets before 1199184111732248575
...1199 tweets downloaded so far
getting tweets before 1199045258346221568
...1399 tweets downloaded so far
getting tweets before 1198650571894534143
...1599 tweets downloaded so far
getting tweets before 1198098690394742784
...1799 tweets downloaded so far
getting tweets before 1197657755588403200
...1999 tweets downloaded so far
getting tweets before 1197320962015997952


...

/usr/lib/python3/dist-packages/simplejson/decoder.py:400: ResourceWarning:

unclosed <ssl.SSLSocket fd=74, family=AddressFamily.AF_INET, type=2049, proto=6, laddr=('128.238.147.119', 34464), raddr=('104.244.42.130', 443)>

/usr/lib/python3/dist-packages/simplejson/decoder.py:400: ResourceWarning:

unclosed <ssl.SSLSocket fd=41, family=AddressFamily.AF_INET, type=2049, proto=6, laddr=('128.238.147.119', 34468), raddr=('104.244.42.130', 443)>

/usr/lib/python3/dist-packages/simplejson/decoder.py:400: ResourceWarning:

unclosed <ssl.SSLSocket fd=39, family=AddressFamily.AF_INET, type=2049, proto=6, laddr=('128.238.147.119', 34470), raddr=('104.244.42.130', 443)>

/usr/lib/python3/dist-packages/simplejson/decoder.py:400: ResourceWarning:

unclosed <ssl.SSLSocket fd=45, family=AddressFamily.AF_INET, type=2049, proto=6, laddr=('128.238.147.119', 34474), raddr=('104.244.42.130', 443)>

/usr/lib/python3/dist-packages/simplejson/decoder.py:400: ResourceWarning:

unclosed <ssl.SSLSocket fd=46, f

...2598 tweets downloaded so far
getting tweets before 1196269982944047104
...2797 tweets downloaded so far
getting tweets before 1195844918859771904
...2997 tweets downloaded so far
getting tweets before 1195462178469556224
...3196 tweets downloaded so far
getting tweets before 1195076995043987455
...3225 tweets downloaded so far
getting tweets before 1194988389210238977
...3225 tweets downloaded so far


/usr/lib/python3.6/urllib/parse.py:825: ResourceWarning:

unclosed <ssl.SSLSocket fd=45, family=AddressFamily.AF_INET, type=2049, proto=6, laddr=('128.238.147.119', 34540), raddr=('104.244.42.130', 443)>

/usr/local/lib/python3.6/dist-packages/tweepy/binder.py:222: ResourceWarning:

unclosed <ssl.SSLSocket fd=46, family=AddressFamily.AF_INET, type=2049, proto=6, laddr=('128.238.147.119', 34542), raddr=('104.244.42.130', 443)>

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: ResourceWarning:

unclosed file <_io.BufferedWriter name='../data/harassers_from_targetted/SueEdelman.p'>



getting tweets before 1200902633239662591
...400 tweets downloaded so far
getting tweets before 1200306786479177727
...600 tweets downloaded so far
getting tweets before 1199762263223062527
...800 tweets downloaded so far
getting tweets before 1199078113218482176
...1000 tweets downloaded so far
getting tweets before 1198633087552253953
...1199 tweets downloaded so far
getting tweets before 1198318364721942527
...1399 tweets downloaded so far
getting tweets before 1197895958966652927
...1599 tweets downloaded so far
getting tweets before 1197231816253394943
...1799 tweets downloaded so far
getting tweets before 1196422617810386944
...1999 tweets downloaded so far
getting tweets before 1195790303560470527
...2199 tweets downloaded so far
getting tweets before 1194842135473688575
...2399 tweets downloaded so far
getting tweets before 1193948178199896064
...2597 tweets downloaded so far
getting tweets before 1192618210593402879
...2797 tweets downloaded so far
getting tweets before 119174

/usr/local/lib/python3.6/dist-packages/tweepy/models.py:44: ResourceWarning:

unclosed <ssl.SSLSocket fd=62, family=AddressFamily.AF_INET, type=2049, proto=6, laddr=('128.238.147.119', 34578), raddr=('104.244.42.130', 443)>

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: ResourceWarning:

unclosed file <_io.BufferedWriter name='../data/harassers_from_targetted/TheJewbyrd7777.p'>



In [ ]:
for t in tweets:
    print(t.text)